In [5]:
import os
import pandas as pd
import math
from sklearn.feature_selection import SelectKBest, f_classif, f_regression
from sklearn.preprocessing import LabelEncoder

def apply_selectkbest(data, target_column, k):
    """
    Aplica SelectKBest para selecionar as melhores variáveis em relação à coluna alvo.

    Args:
        data (pd.DataFrame): Dataset original.
        target_column (str): Nome da coluna alvo.
        k (float or int): Porcentagem (0-1) ou número de variáveis a selecionar.

    Returns:
        pd.DataFrame: Dataset com as variáveis selecionadas e a coluna alvo.
    """
    # Se a coluna alvo for categórica, usa f_classif; caso contrário, usa f_regression
    if data[target_column].dtype == 'object':
        data[target_column] = LabelEncoder().fit_transform(data[target_column])
        score_func = f_classif
    else:
        score_func = f_regression

    # Separar variáveis independentes e dependente
    X = data.drop(columns=[target_column])
    y = data[target_column]

    # Calcular o valor de k com base na porcentagem
    if isinstance(k, float) and 0 < k <= 1:
        k = max(1, math.floor(k * X.shape[1]))  # Arredonda para baixo e garante pelo menos 1

    print(f"Selecionando as top {k} variáveis...")

    # Aplicar SelectKBest
    selector = SelectKBest(score_func=score_func, k=k)
    X_new = selector.fit_transform(X, y)

    # Identificar colunas selecionadas
    selected_columns = X.columns[selector.get_support()]

    print(f"Variáveis selecionadas: {list(selected_columns)}")

    # Reconstruir o DataFrame com as variáveis selecionadas e a coluna alvo
    new_data = pd.DataFrame(X_new, columns=selected_columns)
    new_data[target_column] = y.values

    return new_data

def get_next_folder_number(output_root):
    """
    Determina o próximo número de pasta disponível no diretório de saída.

    Args:
        output_root (str): Diretório raiz para salvar as pastas processadas.

    Returns:
        int: O próximo número disponível para a pasta de saída.
    """
    existing_folders = [
        folder for folder in os.listdir(output_root) if os.path.isdir(os.path.join(output_root, folder))
    ]
    numbered_folders = [
        int(folder.split("-")[0]) for folder in existing_folders if folder.split("-")[0].isdigit()
    ]
    return max(numbered_folders, default=0) + 1

def process_dataset_in_folder(input_dir, output_dir, k):
    """
    Processa os datasets em uma pasta e salva os resultados em outra pasta correspondente.

    Args:
        input_dir (str): Caminho do diretório de entrada.
        output_dir (str): Caminho do diretório de saída.
        k (float or int): Porcentagem (0-1) ou número de variáveis a selecionar.
    """
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(input_dir):
        if filename.endswith(".txt"):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)

            print(f"Processando arquivo: {input_path}")
            try:
                data = pd.read_csv(input_path, delimiter=",", skipinitialspace=True)
                target_column = data.columns[-1]  # Assume que a última coluna é a coluna alvo

                # Aplicar SelectKBest
                new_data = apply_selectkbest(data, target_column, k)

                # Salvar o novo dataset
                new_data.to_csv(output_path, sep=",", index=False)
                print(f"Arquivo processado e salvo em: {output_path}")

            except Exception as e:
                print(f"Erro ao processar {filename}: {e}")

def process_multiple_folders(input_root, output_root, k):
    """
    Processa múltiplas pastas dentro do diretório de entrada e salva os resultados diretamente
    no diretório de saída com nomes incrementados.

    Args:
        input_root (str): Diretório raiz contendo as pastas de entrada.
        output_root (str): Diretório raiz para salvar as pastas processadas.
        k (float or int): Porcentagem (0-1) ou número de variáveis a selecionar.
    """
    next_folder_number = get_next_folder_number(output_root)

    for folder_name in os.listdir(input_root):
        input_folder_path = os.path.join(input_root, folder_name)
        output_folder_name = f"{next_folder_number}-saida"
        output_folder_path = os.path.join(output_root, output_folder_name)

        if os.path.isdir(input_folder_path):
            print(f"\nProcessando pasta {folder_name} para {output_folder_name}...")
            process_dataset_in_folder(input_folder_path, output_folder_path, k=k)
            next_folder_number += 1

# Diretórios de entrada e saída
input_root_directory = r"C:\\Users\\CALEO\\OneDrive - Hexagon\\Documents\\GitHub\\Software_effort_estimation\\proposal\\algorithms\\abordagens\\output"
output_root_directory = r"C:\\Users\\CALEO\\OneDrive - Hexagon\\Documents\\GitHub\\Software_effort_estimation\\proposal\\algorithms\\abordagens\\output3"

# Processar arquivos usando SelectKBest para selecionar 25% das variáveis
top_percentage = 0.50  # 50% das variáveis
process_multiple_folders(input_root_directory, output_root_directory, k=top_percentage)



Processando pasta 0-saida para 15-saida...
Processando arquivo: C:\\Users\\CALEO\\OneDrive - Hexagon\\Documents\\GitHub\\Software_effort_estimation\\proposal\\algorithms\\abordagens\\output\0-saida\cocomo81.txt
Selecionando as top 8 variáveis...
Variáveis selecionadas: ['rely', 'data', 'time', 'turn', 'acap', 'pcap', 'modp', 'loc']
Arquivo processado e salvo em: C:\\Users\\CALEO\\OneDrive - Hexagon\\Documents\\GitHub\\Software_effort_estimation\\proposal\\algorithms\\abordagens\\output3\15-saida\cocomo81.txt
Processando arquivo: C:\\Users\\CALEO\\OneDrive - Hexagon\\Documents\\GitHub\\Software_effort_estimation\\proposal\\algorithms\\abordagens\\output\0-saida\encoded_china.txt
Selecionando as top 10 variáveis...
Variáveis selecionadas: ['AFP', 'Input', 'Output', 'Enquiry', 'File', 'Interface', 'Added', 'PDR_UFP', 'Duration', 'N_effort']
Arquivo processado e salvo em: C:\\Users\\CALEO\\OneDrive - Hexagon\\Documents\\GitHub\\Software_effort_estimation\\proposal\\algorithms\\abordagens\

C:\Users\CALEO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\feature_selection\_univariate_selection.py:378: RuntimeWarning: invalid value encountered in sqrt
  X_norms = np.sqrt(row_norms(X.T, squared=True) - n_samples * X_means**2)


Arquivo processado e salvo em: C:\\Users\\CALEO\\OneDrive - Hexagon\\Documents\\GitHub\\Software_effort_estimation\\proposal\\algorithms\\abordagens\\output3\23-saida\2-saida_encoded_maxwell.txt

Processando pasta 5-saida para 24-saida...
Processando arquivo: C:\\Users\\CALEO\\OneDrive - Hexagon\\Documents\\GitHub\\Software_effort_estimation\\proposal\\algorithms\\abordagens\\output\5-saida\cocomo81.txt
Selecionando as top 10 variáveis...
Variáveis selecionadas: ['rely', 'data', 'turn', 'pcap', 'modp', 'loc', 'Generated_Var_1', 'Generated_Var_2', 'Generated_Var_3', 'Generated_Var_4']
Arquivo processado e salvo em: C:\\Users\\CALEO\\OneDrive - Hexagon\\Documents\\GitHub\\Software_effort_estimation\\proposal\\algorithms\\abordagens\\output3\24-saida\cocomo81.txt
Processando arquivo: C:\\Users\\CALEO\\OneDrive - Hexagon\\Documents\\GitHub\\Software_effort_estimation\\proposal\\algorithms\\abordagens\\output\5-saida\encoded_china.txt
Selecionando as top 13 variáveis...
Variáveis selecionad